In [9]:
import numpy as np
import os
import re
import pandas as pd
from abby_curvature import scalar_curvature_est
from diffusion_curvature import diffusion_curvature

In [16]:
def aggr_res(i):
    data_dir = f"torus_noise_0.{i}/"
    # Initialize an empty list to store dataframes
    df_list = []

    # Regular expression to match the filenames and extract IDs
    pattern = re.compile(r"curvature_from_(\d+)_to_(\d+)\.csv")

    # Iterate over all files in the directory (replace '.' with your directory path)
    for filename in os.listdir(data_dir):
        match = pattern.match(filename)
        if match:
            # Extract the range of ids from the filename
            start_id, end_id = map(int, match.groups())

            # Read the csv file into a DataFrame
            df = pd.read_csv(data_dir + filename, header=None)
            
            # Create a new column for the ids
            df['id'] = range(start_id, end_id)
            
            # Append the DataFrame to the list
            df_list.append(df)

    # Concatenate all DataFrames in the list
    final_df = pd.concat(df_list, ignore_index=True)
    final_df = final_df.set_index('id').sort_index()
    # Save the final DataFrame to a new csv file
    # final_df.to_csv('curvature_torus_ratio_4_combined.csv', index=False)
    # np.savetxt("curvature_torus_ratio_4.0_combined.csv", final_df.values)
    # Show the head of the final DataFrame as a sample

    assert final_df.shape[0] == 5000
    # print(final_df.shape[0])
    torus = np.loadtxt(data_dir + 'point_cloud.csv', delimiter=',')
    torus_noiseless = np.loadtxt(data_dir + 'point_cloud_noiseless.csv', delimiter=',')
    true_torus = np.loadtxt(data_dir + "K.csv")
    our_torus = final_df.values.flatten()
    # dc_torus = diffusion_curvature(torus)
    # sce_torus = scalar_curvature_est(2, torus)
    # ab_torus = np.array(sce_torus.estimate(rmax=np.pi/2))
    non_na_id = ~np.isnan(our_torus)
    print(i)
    print("ours:", np.corrcoef(-our_torus[non_na_id], true_torus[non_na_id])[0, 1])
    # print("diffusion:", np.corrcoef(dc_torus, true_torus)[0, 1])
    # print("abby:", np.corrcoef(ab_torus, true_torus)[0, 1])
    return {
        "noise": i * 0.1,
        "ours": np.corrcoef(-our_torus[non_na_id], true_torus[non_na_id])[0, 1],
        "non-na percentage": non_na_id.mean()
        }
        # "diffusion": np.corrcoef(dc_torus, true_torus)[0, 1],
        # "abby": np.corrcoef(ab_torus, true_torus)[0, 1]
    # }

In [17]:
# res_df = pd.DataFrame([aggr_res(i) for i in range(10)])
res_df = pd.DataFrame([aggr_res(i) for i in range(9)])

0
ours: 0.9077682680676546
1
ours: 0.8381280411184178
2
ours: 0.37979938623327253
3
ours: 0.18660555836089163
4
ours: 0.0963262912298727
5
ours: 0.1433454140374379
6
ours: 0.10033093565493838
7
ours: 0.07014304946721052
8
ours: 0.02717263688298611


In [ ]:
res_df.to_csv("noise_aggregates.csv")

In [18]:
res_df

,noise,ours,non-na percentage
0,0.0,0.907768,0.9882
1,0.1,0.838128,0.9942
2,0.2,0.379799,0.4802
3,0.3,0.186606,0.3228
4,0.4,0.096326,0.2606
5,0.5,0.143345,0.2424
6,0.6,0.100331,0.2118
7,0.7,0.070143,0.1642
8,0.8,0.027173,0.1142
